**Kernel Method challenge**

# Initialization and data preparation

* Data are loaded and standardized with zero mean and unit variance


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Load and standardize features by removing the mean and scaling to unit variance
StandardScaler = lambda df: (df-df.mean())/df.std()

XTrain0 = pd.read_csv('./data/Xtr0_mat100.csv', sep=' ', header=None)
XTrain1 = pd.read_csv('./data/Xtr1_mat100.csv', sep=' ', header=None)
XTrain2 = pd.read_csv('./data/Xtr2_mat100.csv', sep=' ', header=None)


XTest0 = (pd.read_csv('./data/Xte0_mat100.csv', sep=' ', header=None)-XTrain0.mean())/XTrain0.std()
XTest1 = (pd.read_csv('./data/Xte1_mat100.csv', sep=' ', header=None)-XTrain1.mean())/XTrain1.std()
XTest2 = (pd.read_csv('./data/Xte2_mat100.csv', sep=' ', header=None)-XTrain2.mean())/XTrain2.std()

YTrain0 = pd.read_csv('./data/Ytr0.csv', usecols = ['Bound'])
YTrain1 = pd.read_csv('./data/Ytr1.csv', usecols = ['Bound'])
YTrain2 = pd.read_csv('./data/Ytr2.csv', usecols = ['Bound'])


XTrain0 = StandardScaler(XTrain0)
XTrain1 = StandardScaler(XTrain1)
XTrain2 = StandardScaler(XTrain2)

# Baseline Model: Ridge Regression on the numerical features (from scratch)


This section contains:

* The implementation of the Ridge Regression from scratch
* A comparison of the results with the one provided by the sklearn implementation. Such test is performed using part of the training set for training and the rest for testing and to compute performances accuracy. 


* https://github.com/akaashagarwal/ridge-regression 

In [4]:
class RidgeRegression:
    """Class to fit a ridge regression model on a given training set, and make predictions on data.
    Specifically, the cost function with L_2 regularization is assumed to be:
        ||w - Xw||^2_2 + alpha * ||w||2_2
    where,
        w = Model weights,
        X = Data design matrix,
        alpha = regularization parameter.
    The class uses batch gradient descent to optimize the model weights.
    """

    def __init__(self, learning_rate: float, reg_strength: float, max_iter: int) -> None:
        """Initialize.
        Args:
            learning_rate (float): Learning rate for gradient descent.
            reg_strength (float): Regularization parameter, to control the bias-variance tradeoff.
            max_iter (int): Number of iterations to run gradient descent.
        """
        self.learn_rate = learning_rate
        self.reg_strength = reg_strength
        self.max_iter = max_iter
        self._weights: np.array
        self.design_matrix: np.array
        self.y_train: np.array
        self.train_size: int
        self.num_feats: int

    @property
    def weights(self):
        """Return ridge regression model weights."""
        return self._weights

    def fit(self, x_train, y_train) -> None:
        """Fit model weights to training data."""
        self.train_size, self.num_feats = x_train.shape[0], x_train.shape[1]
        self.design_matrix = np.append(np.ones(self.train_size).reshape(
            -1, 1), x_train, axis=1)  # Add one for the intercept term for all training examples.
        self.y_train = y_train.to_numpy()
        self._weights = np.zeros(self.num_feats + 1)  # +1 for the intercept.

        self.optimize_weights()

    def optimize_weights(self) -> None:
        """Optimize model weights for self.max_iter iterations."""
        for _ in range(self.max_iter):
            self.update_step()

    def update_step(self) -> None:
        """
            Update model weights with batch gradient descent step.
            Recall, the update statement for a model weight theta_k is:
                theta_k = theta_k - (learn_rate * J_theta_k)
                where J_theta_k = cost function
                      J_theta_k = (2 / train_size) * ( ((y_hat - y_real) * x_k) + (alpha * theta_k^2))
        """
        y_hat = (self._weights * self.design_matrix).sum(axis=1)
        errors = (y_hat - self.y_train).reshape(-1, 1)

        j_theta = (2 / self.train_size) * ((errors * self.design_matrix).sum(axis=0) +
                                           (self.reg_strength * self._weights))
        step = self.learn_rate * j_theta

        self._weights = self._weights - step.reshape(-1)

    def predict(self, x_test):
        """Make predictions for x_test using trained model.
        Args:
            x_test (np.array): Input test data set.
        Returns:
            np.array: Predictions for x_test.
        """
        test_size = x_test.shape[0]
        x_test = np.append(np.ones(test_size).reshape(-1, 1), x_test, axis=1)

        return (self._weights * x_test).sum(axis=1)

In [27]:
# Test on the training data of the Ridge Regression from scratch 
RidgeRegressor = RidgeRegression(learning_rate=0.1, reg_strength=10, max_iter=1000)
RidgeRegressor.fit(XTrain0[:1900], YTrain0['Bound'][:1900])
predictions = RidgeRegressor.predict(XTrain0[1900:])
y_true = np.array(YTrain0['Bound'][1900:])
y_pred = np.array(np.rint(y_pred))
abs_errors = np.abs((y_true - y_pred))
print('RR from scratch: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))

from sklearn.linear_model import Ridge
sklearnRegressor = Ridge(alpha=10, solver='sag', max_iter=1000)
model = sklearnRegressor.fit(XTrain0[:1900], YTrain0['Bound'][:1900])
predictions = model.predict(XTrain0[1900:])

y_true = np.array(YTrain0['Bound'][1900:])
y_pred = np.array(np.rint(y_pred))

abs_errors = np.abs((y_true - y_pred))
print('Sklearn implementation: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))


RR from scratch: Error on the 5% of the training set: 0.38
Sklearn implementation: Error on the 5% of the training set: 0.38


In [23]:
# First submission using Ridge Regression

RidgeRegressor = RidgeRegression(learning_rate=0.1, reg_strength=10, max_iter=1000)
RidgeRegressor.fit(XTrain0, YTrain0['Bound'])
predictions0 = RidgeRegressor.predict(XTest0)
predictions0 = np.rint(predictions0).astype(int)

df0 = pd.DataFrame({'Id': np.arange(1000), 'Bound': predictions0})

RidgeRegressor = RidgeRegression(learning_rate=0.1, reg_strength=10, max_iter=1000)
RidgeRegressor.fit(XTrain1, YTrain1['Bound'])
predictions1 = RidgeRegressor.predict(XTest1)
predictions1 = np.rint(predictions1).astype(int)

df1 = pd.DataFrame({'Id': np.arange(1000,2000), 'Bound': predictions1})


RidgeRegressor = RidgeRegression(learning_rate=0.1, reg_strength=10, max_iter=1000)
RidgeRegressor.fit(XTrain2, YTrain2['Bound'])
predictions2 = RidgeRegressor.predict(XTest2)
predictions2 = np.rint(predictions2).astype(int)

df2 = pd.DataFrame({'Id': np.arange(2000,3000), 'Bound': predictions2})


dfResult = pd.concat([df0,df1,df2])
dfResult.to_csv('./data/submissionRidgeRegression.csv', index=False)

# Kernel Ridge Regression on the numerical features

This section contains:

* The implementation of the Kernel Ridge Regression from scratch, with the generation of the Kaggle submission file. 
* A coarse grid searches on the parameters (the regularisation parameter $\lambda$ and $\sigma$ the std of the gaussian Kernel)
* A comparison of the results with the one provided by the sklearn implementation. Such test is performed using part of the training set for training and the rest for testing and to compute performances accuracy. 

In [75]:
# Kernel Ridge regression and creation of the Kaggle submission file

from scipy.spatial.distance import pdist, squareform

def gaussian_kernel(z):
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*z**2)

def distance_matrix(A, B, squared=False):
    """
    Compute all pairwise distances between vectors in A and B.

    Parameters
    ----------
    A : np.array
        shape should be (M, K)
    B : np.array
        shape should be (N, K)

    Returns
    -------
    D : np.array
        A matrix D of shape (M, N).  Each entry in D i,j represnets the
        distance between row i in A and row j in B.

    See also
    --------
    A more generalized version of the distance matrix is available from
    scipy (https://www.scipy.org) using scipy.spatial.distance_matrix,
    which also gives a choice for p-norm.
    """
    M = A.shape[0]
    N = B.shape[0]

    assert A.shape[1] == B.shape[1], f"The number of components for vectors in A \
        {A.shape[1]} does not match that of B {B.shape[1]}!"

    A_dots = (A*A).sum(axis=1).reshape((M,1))*np.ones(shape=(1,N))
    B_dots = (B*B).sum(axis=1)*np.ones(shape=(M,1))
    D_squared =  A_dots + B_dots -2*A.dot(B.T)

    if squared == False:
        zero_mask = np.less(D_squared, 0.0)
        D_squared[zero_mask] = 0.0
        return np.sqrt(D_squared)

    return D_squared
    
    
sigma = 1.233

# Computation of the three Kernel Matrix Ki
K0 = gaussian_kernel(distance_matrix(XTrain0.to_numpy(), XTrain0.to_numpy())/sigma)
K1 = gaussian_kernel(distance_matrix(XTrain1.to_numpy(), XTrain1.to_numpy())/sigma)
K2 = gaussian_kernel(distance_matrix(XTrain2.to_numpy(), XTrain2.to_numpy())/sigma)

lambdaParameter=1

n = XTrain0.shape[0]
alpha0 = np.linalg.solve(K0+lambdaParameter*n*np.eye(XTrain0.shape[0]),2*YTrain0.to_numpy()-1)

n = XTrain1.shape[0]
alpha1 = np.linalg.solve(K1+lambdaParameter*n*np.eye(XTrain1.shape[0]),2*YTrain1.to_numpy()-1)

n = XTrain2.shape[0]
alpha2 = np.linalg.solve(K2+lambdaParameter*n*np.eye(XTrain2.shape[0]),2*YTrain2.to_numpy()-1)

# Predictions using the previsously computed weights (i.e the optimal function ion the RKHS)
predictions0 = alpha0.T@gaussian_kernel(distance_matrix(XTrain0.to_numpy(), XTest0.to_numpy())/sigma)
predictions1 = alpha1.T@gaussian_kernel(distance_matrix(XTrain1.to_numpy(), XTest1.to_numpy())/sigma)
predictions2 = alpha2.T@gaussian_kernel(distance_matrix(XTrain2.to_numpy(), XTest2.to_numpy())/sigma)

# Predictions where -1 or +1, we set them back to 0, 1 for the submission.
predictions0 = (1+np.sign(predictions0))/2
predictions1 = (1+np.sign(predictions1))/2
predictions2 = (1+np.sign(predictions2))/2

# Creation of the Kaggle submission file
df0 = pd.DataFrame({'Id': np.arange(1000), 'Bound': predictions0.squeeze().astype(int)})
df1 = pd.DataFrame({'Id': np.arange(1000,2000), 'Bound': predictions1.squeeze().astype(int)})
df2 = pd.DataFrame({'Id': np.arange(2000,3000), 'Bound': predictions2.squeeze().astype(int)})
dfResult = pd.concat([df0,df1,df2])
dfResult.to_csv('./data/submissionKernelRidgeRegression.csv', index=False)

In [51]:
# Comparison of our implementation and the Sklearn one 

def gaussian_kernel(z):
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*z**2)

# Computation of the three Kernel Matrix Ki
K0 = gaussian_kernel(distance_matrix(XTrain0[:1800].to_numpy(), XTrain0[:1800].to_numpy())/sigma)

lambdaParameter=1
n = XTrain0[:1800].shape[0]
alpha0 = np.linalg.solve(K0+lambdaParameter*n*np.eye(XTrain0[:1800].shape[0]),2*YTrain0[:1800].to_numpy()-1)

# Predictions using the previsously computed weights (i.e the optimal function ion the RKHS)
predictions0 = alpha0.T@gaussian_kernel(distance_matrix(XTrain0[:1800].to_numpy(), XTrain0[1800:].to_numpy())/sigma)

y_true = np.array(YTrain0['Bound'][1800:])
y_pred = (1+np.sign(predictions0))/2

abs_errors = np.abs((y_true - y_pred))
print('Kernel Ridge Regression from scratch implementation: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))


# Comparison with the sklearn implementation 
from sklearn.kernel_ridge import KernelRidge
def gaussian_kernel(x,y):
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*np.linalg.norm(x-y)**2)
    
    
clf = KernelRidge(alpha=1.0, kernel=gaussian_kernel)
clf.fit(XTrain0[:1800], 2*YTrain0['Bound'][:1800]-1)
predictions = clf.predict(XTrain0[1800:])
y_true = np.array(YTrain0['Bound'][1800:])
y_pred = (1+np.sign(predictions))/2

abs_errors = np.abs((y_true - y_pred))
print('Sklearn implementation: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))

Kernel Ridge Regression from scratch implementation: Error on the 5% of the training set: 0.385
Sklearn implementation: Error on the 5% of the training set: 0.385


In [ ]:
# Grid search on the parameters of the Kernel Ridge Regression

def gaussian_kernel(z):
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*z**2)


for sigm in np.linspace(1.1,1.3,10):#[0.4, 0.45, 0.5, 0.55, 0.6]:
    for lambda_param in [0.1, 1., 10.]:
        print('\nLambda: {} sigma: {}'.format(lambda_param,sigm))

        # Computation of the three Kernel Matrix Ki
        K0 = gaussian_kernel(distance_matrix(XTrain0[:1800].to_numpy(), XTrain0[:1800].to_numpy())/sigm)
        n = XTrain0[:1800].shape[0]
        alpha0 = np.linalg.solve(K0+lambda_param*n*np.eye(XTrain0[:1800].shape[0]),2*YTrain0[:1800].to_numpy()-1)
        # Predictions using the previsously computed weights (i.e the optimal function ion the RKHS)
        predictions0 = alpha0.T@gaussian_kernel(distance_matrix(XTrain0[:1800].to_numpy(), XTrain0[1800:].to_numpy())/sigm)
        y_true = np.array(YTrain0['Bound'][1800:])
        y_pred = (1+np.sign(predictions0))/2
        abs_errors0 = np.abs((y_true - y_pred))
        #print('XTRAIN0: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))
        
        # Computation of the three Kernel Matrix Ki
        K0 = gaussian_kernel(distance_matrix(XTrain1[:1800].to_numpy(), XTrain1[:1800].to_numpy())/sigm)
        n = XTrain1[:1800].shape[0]
        alpha0 = np.linalg.solve(K0+lambda_param*n*np.eye(XTrain1[:1800].shape[0]),2*YTrain1[:1800].to_numpy()-1)
        # Predictions using the previsously computed weights (i.e the optimal function ion the RKHS)
        predictions0 = alpha0.T@gaussian_kernel(distance_matrix(XTrain1[:1800].to_numpy(), XTrain1[1800:].to_numpy())/sigm)
        y_true = np.array(YTrain1['Bound'][1800:])
        y_pred = (1+np.sign(predictions0))/2
        abs_errors1 = np.abs((y_true - y_pred))
        #print('XTRAIN1: Error on the 5% of the training set: {}'.format(np.mean(abs_errors)))

        # Computation of the three Kernel Matrix Ki
        K0 = gaussian_kernel(distance_matrix(XTrain2[:1800].to_numpy(), XTrain2[:1800].to_numpy())/sigm)
        n = XTrain1[:1800].shape[0]
        alpha0 = np.linalg.solve(K0+lambda_param*n*np.eye(XTrain2[:1800].shape[0]),2*YTrain2[:1800].to_numpy()-1)
        # Predictions using the previsously computed weights (i.e the optimal function ion the RKHS)
        predictions0 = alpha0.T@gaussian_kernel(distance_matrix(XTrain2[:1800].to_numpy(), XTrain2[1800:].to_numpy())/sigm)
        y_true = np.array(YTrain2['Bound'][1800:])
        y_pred = (1+np.sign(predictions0))/2
        abs_errors2 = np.abs((y_true - y_pred))
        print('Mean error on the 5% of the 3 training set: {}'.format(np.mean(list(abs_errors0.squeeze())+list(abs_errors1.squeeze())+list(abs_errors2.squeeze()))))


# Support Vector Machine 